## --- Day 10: Syntax Scoring ---

You ask the submarine to determine the best route out of the deep-sea cave, but it only replies:

Syntax error in navigation subsystem on line: all of them
All of them?! The damage is worse than you thought. You bring up a copy of the navigation subsystem (your puzzle input).

The navigation subsystem syntax is made of several lines containing chunks. There are one or more chunks on each line, and chunks contain zero or more other chunks. Adjacent chunks are not separated by any delimiter; if one chunk stops, the next chunk (if any) can immediately start. Every chunk must open and close with one of four legal pairs of matching characters:

If a chunk opens with (, it must close with ).
If a chunk opens with [, it must close with ].
If a chunk opens with {, it must close with }.
If a chunk opens with <, it must close with >.
So, () is a legal chunk that contains no other chunks, as is []. More complex but valid chunks include ([]), {()()()}, <([{}])>, [<>({}){}[([])<>]], and even (((((((((()))))))))).

Some lines are incomplete, but others are corrupted. Find and discard the corrupted lines first.

A corrupted line is one where a chunk closes with the wrong character - that is, where the characters it opens and closes with do not form one of the four legal pairs listed above.

Examples of corrupted chunks include (], {()()()>, (((()))}, and <([]){()}[{}]). Such a chunk can appear anywhere within a line, and its presence causes the whole line to be considered corrupted.

For example, consider the following navigation subsystem:

    [({(<(())[]>[[{[]{<()<>>
    [(()[<>])]({[<{<<[]>>(
    {([(<{}[<>[]}>{[]{[(<()>
    (((({<>}<{<{<>}{[]{[]{}
    [[<[([]))<([[{}[[()]]]
    [{[{({}]{}}([{[{{{}}([]
    {<[[]]>}<{[{[{[]{()[[[]
    [<(<(<(<{}))><([]([]()
    <{([([[(<>()){}]>(<<{{
    <{([{{}}[<[[[<>{}]]]>[]]
    
Some of the lines aren't corrupted, just incomplete; you can ignore these lines for now. The remaining five lines are corrupted:

{([(<{}[<>[]}>{[]{[(<()> - Expected ], but found } instead.
[[<[([]))<([[{}[[()]]] - Expected ], but found ) instead.
[{[{({}]{}}([{[{{{}}([] - Expected ), but found ] instead.
[<(<(<(<{}))><([]([]() - Expected >, but found ) instead.
<{([([[(<>()){}]>(<<{{ - Expected ], but found > instead.
Stop at the first incorrect closing character on each corrupted line.

Did you know that syntax checkers actually have contests to see who can get the high score for syntax errors in a file? It's true! To calculate the syntax error score for a line, take the first illegal character on the line and look it up in the following table:

): 3 points.
]: 57 points.
}: 1197 points.
>: 25137 points.

In the above example, an illegal ) was found twice (2*3 = 6 points), an illegal ] was found once (57 points), an illegal } was found once (1197 points), and an illegal > was found once (25137 points). So, the total syntax error score for this file is 6+57+1197+25137 = 26397 points!

Find the first illegal character in each corrupted line of the navigation subsystem. What is the total syntax error score for those errors?

In [2]:
with open("day10.txt") as f:
    lines = [line.strip() for line in f.readlines()]
lines[:3]  # so a list (array in C#)

['(((<(([{{((<[[<><>]<()[]>]{[{}()]<(){}>}>[<([]{})>])(<<(<><>>[<><>]>(<{}()><(){}>)>[(([])<[]()>)])){<((<{}()',
 '{(({({<[[[{[{({}{})[[]{}]}[{(){}}((){})]][{((){}){[]{}}}<(()())<<><>>>]}]]]{([<{<<()()>([][])>({[]()}[{}<',
 '[[{<(([<<{(<[[()()]]{[[]()]<{}>}><{{{}()}<<>[]>><(())[()[]]>>)([{[[][]][()()]}{{()()}}](<{{}()}>))}[([{<<']

In [4]:
# creating a dictionary (Map structure for answers)
matches = {"]":"[", ")":"(","}":"{",">":"<"}
matches

{']': '[', ')': '(', '}': '{', '>': '<'}

In [7]:
matches.keys()

dict_keys([']', ')', '}', '>'])

In [5]:
scores = {"]":57, ")":3,"}":1197,">":25137}
scores

{']': 57, ')': 3, '}': 1197, '>': 25137}

In [9]:
stack = [] # actually a list but we dont care

In [10]:
stack.pop() # we might need to handle this empty case later on

IndexError: pop from empty list

In [6]:
first = lines[0]
first

'(((<(([{{((<[[<><>]<()[]>]{[{}()]<(){}>}>[<([]{})>])(<<(<><>>[<><>]>(<{}()><(){}>)>[(([])<[]()>)])){<((<{}()'

In [12]:
for c in first:
    if c in matches.keys(): 
        if stack.pop() == matches[c]: # everything is alright with the world we continue
            continue # with the loop
        else:
            print("Corrupted character!!", c)
            print("Score", scores[c])
            break
    else:
        stack.append(c)

Corrupted character!! >
Score 25137


In [13]:
def getScore(line, matches, scores):
    stack = []
    for c in line:
        if c in matches.keys(): 
            if stack.pop() == matches[c]: # everything is alright with the world we continue
                continue # with the loop
            else:
#                 print("Corrupted character!!", c)
#                 print("Score", scores[c])
                return scores[c]
        else:
            stack.append(c)
    return 0 # everything was good

In [15]:
getScore(first, matches, scores)

25137

In [18]:
error_scores = [getScore(line, matches, scores) for line in lines]
error_scores[:15]

[25137, 0, 25137, 25137, 0, 57, 1197, 1197, 25137, 3, 0, 0, 0, 0, 1197]

In [19]:
sum(error_scores)

323613

In [20]:
closers = {value:key for (key,value) in matches.items()}  # reversing the dictionary (Map)
closers

{'[': ']', '(': ')', '{': '}', '<': '>'}

In [21]:
closing_scores = {"]":2, ")":1,"}":3,">":4}

In [34]:
def getClosingScore(parenthesis, closing_scores):
    total = 0
    for c in parenthesis:
        total *= 5
        total += closing_scores[c]
    return total
getClosingScore("}}]])})]", closing_scores)

288957

In [24]:
getClosingScore(")}>]})")

5566

In [32]:
openers_scores = {"(":1,"[":2,"{":3,"<":4}  # my stack will have these unclosed ones so I will count those instead of converting
# because there is 1 to 1 matching between openers and closers
openers_scores

{'(': 1, '[': 2, '{': 3, '<': 4}

In [35]:
def getGoodScore(line, matches):
    stack = []
    for c in line:
        if c in matches.keys(): 
            if stack.pop() == matches[c]: # everything is alright with the world we continue
                continue # with the loop
            else:
#                 print("Corrupted character!!", c)
#                 print("Score", scores[c])
                return 0 # here we return 0 on corruption
        else:
            stack.append(c)
    
    return getClosingScore(stack[::-1], openers_scores)  # i need to reverse the stack because I start with the last to enter 

In [36]:
getGoodScore(first, matches)

0

In [37]:
good_scores = [getGoodScore(line, matches) for line in lines]
good_scores[:5]

[0, 27289760408, 0, 0, 7698571]

In [38]:
# we need to filter out 0 (the corrupted ones)
good_scores_clean = [score for score in good_scores if score != 0]
good_scores_clean[:5]

[27289760408, 7698571, 3071035568, 27919410874, 6832816]

In [39]:
good_scores_sorted = sorted(good_scores_clean)
good_scores_sorted[:5]

[11786, 2540332, 3426748, 3695218, 6832816]

In [40]:
len(good_scores_sorted)

45

In [41]:
median_index = len(good_scores_sorted) // 2 # so basically floor
median_index 

22

In [42]:
median = good_scores_sorted[median_index]
median

3103006161

In [44]:
# lazy way is to use a built in module
import statistics

In [45]:
statistics.median(good_scores_clean)

3103006161